<a href="https://colab.research.google.com/github/jordiyapz/snow-prediction-with-pso-based-mlp/blob/main/prediksi_salju.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediksi Salju

Ditulis oleh:
- Jordi Yaputra (1301180353)
- M Fadhil (1301180263)

In [ ]:
# Using pre-generated data and model instead of regenerating a new one.
USE_PREPROCESSED = True

# Using pre-trained model instead of training new model..
USE_PRETRAINED = True

In [ ]:
from google.colab import files
import os

In [ ]:
# salju_zip = files.upload()
!wget -O salju.zip https://github.com/fadhilfadhil/Malin_Tubes1/blob/main/salju.zip?raw=true
!unzip salju.zip 
!rm salju.zip

--2021-06-09 07:03:21--  https://github.com/fadhilfadhil/Malin_Tubes1/blob/main/salju.zip?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/fadhilfadhil/Malin_Tubes1/raw/main/salju.zip [following]
--2021-06-09 07:03:21--  https://github.com/fadhilfadhil/Malin_Tubes1/raw/main/salju.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fadhilfadhil/Malin_Tubes1/main/salju.zip [following]
--2021-06-09 07:03:21--  https://raw.githubusercontent.com/fadhilfadhil/Malin_Tubes1/main/salju.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response.

In [ ]:
import pickle
import pandas as pd
import numpy as np
import numba as nb
from numba import njit, jit

In [ ]:
if USE_PREPROCESSED:
    with open('salju/imputer.pkl', 'rb') as f:
        imputer = pickle.load(f)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator KNNImputer from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
# import toolkit yang ditulis sendiri
!pip install git+https://github.com/jordiyapz/ai-toolkit.git --upgrade
from jordiyapz_ai_toolkit.util import one_hot_encode, replace_values
from jordiyapz_ai_toolkit.model.pipeline import Pipeline

  Cloning https://github.com/jordiyapz/ai-toolkit.git to /tmp/pip-req-build-qbak60le
  Running command git clone -q https://github.com/jordiyapz/ai-toolkit.git /tmp/pip-req-build-qbak60le
  Created wheel for jordiyapz-ai-toolkit: filename=jordiyapz_ai_toolkit-1.0.0-cp37-none-any.whl size=9938 sha256=069fefa547e346edd91fafd2859b96c59db6f213aabbdf0517c1e78105338c92
  Stored in directory: /tmp/pip-ephem-wheel-cache-89al4vfi/wheels/8a/ec/ae/e75aad71867990fbd7bf5d87dc9119818393054f7433d8222f
Successfully built jordiyapz-ai-toolkit


## Toolkit

In [ ]:
import numba as nb
from numba import njit, jit
from numba.experimental import jitclass

import time
import numpy as np

In [ ]:
import pdb

### Utilities

#### Numba Numpy Helper

In [ ]:
X = np.random.rand(6, 3) * 10 - 5

In [ ]:
@njit
def clip_np(a, a_min, a_max, out=None):
  m_max = a < a_max
  m_min = a > a_min

  mask = m_max * m_min
  not_mask = 1 - mask
  
  return a * mask + (m_max * a_min + m_min * a_max) * not_mask

display(X)
clip_np(X, -3, 2.5)

array([[ 0.18461034,  3.83650717, -1.47633618],
       [-0.28254311,  1.97013632, -2.05264315],
       [ 0.17698824,  3.38004073,  3.2030965 ],
       [-4.07068549, -1.95781636, -1.08111618],
       [ 1.9319337 ,  2.36150286,  1.14337151],
       [-3.61979668,  0.13608549, -4.04620403]])

array([[ 0.18461034,  2.5       , -1.47633618],
       [-0.28254311,  1.97013632, -2.05264315],
       [ 0.17698824,  2.5       ,  2.5       ],
       [-3.        , -1.95781636, -1.08111618],
       [ 1.9319337 ,  2.36150286,  1.14337151],
       [-3.        ,  0.13608549, -3.        ]])

In [ ]:
@nb.njit
def _apply_along_axis_0(func1d, arr, out):
    """Like calling func1d(arr, axis=0, out=out). Require arr to be 2d or bigger."""
    ndim = arr.ndim
    if ndim < 2:
        raise RuntimeError("_apply_along_axis_0 requires 2d array or bigger")
    elif ndim == 2:  # 2-dimensional case
        for i in range(len(out)):
            out[i] = func1d(arr[:, i])
    else:  # higher dimensional case
        for i, out_slice in enumerate(out):
            _apply_along_axis_0(func1d, arr[:, i], out_slice)

@nb.njit
def apply_along_axis_0(func1d, arr):
    """Like calling func1d(arr, axis=0)"""
    if arr.size == 0:
        raise RuntimeError("Must have arr.size > 0")
    ndim = arr.ndim
    if ndim == 0:
        raise RuntimeError("Must have ndim > 0")
    elif 1 == ndim:
        return func1d(arr)
    else:
        result_shape = arr.shape[1:]
        out = np.empty(result_shape, arr.dtype)
        _apply_along_axis_0(func1d, arr, out)
        return out

In [ ]:
@nb.njit
def np_argmax_axis_0(arr):
  return apply_along_axis_0(np.argmax, arr)

In [ ]:
def timeit(func):
  def func_(*args):
    ts = time.time()
    ret_val = func(*args)
    te = time.time()
    print(f'ETA: {te-ts} sec')
    return ret_val
  return func_

In [ ]:
@njit
def cast_to_int_1d(array):
  ret = np.zeros_like(array, dtype=np.int32)
  for i in range(array.shape[0]):
    ret[i] = int(array[i])
  return ret

@njit
def cast_to_int_2d(matrix):
  ret = np.zeros_like(matrix, dtype=np.int32)
  for i in range(matrix.shape[0]):
    for j in range(matrix.shape[1]):
      ret[i, j] = int(matrix[i, j])
  return ret

cast_to_int_1d(np.random.rand(5 + 3) * 10 - 5)
cast_to_int_2d(np.random.rand(5, 3) * 10 - 5)

array([[-3,  4, -2],
       [-4,  0,  0],
       [ 1,  4,  0],
       [ 0, -4, -1],
       [ 1, -4,  3]], dtype=int32)

#### Activation Function

In [ ]:
@njit
def relu(Z):
  return np.maximum(Z, 0)

@njit
def sigmoid(Z):
  return 1 / (1 + np.exp(-Z))

@njit
def tanh(Z):
  return np.tanh(Z)
  # return (2 / (1 + np.exp(-2 * Z))) - 1
  # return (np.exp(Z) - np.exp(-Z))/(np.exp(Z) + np.exp(-Z))
  
X = np.array([[2, 3, 4, -2, 1], [3, -4, 0, 1, 1]])
display(relu(X))
display(sigmoid(X))
display(tanh(X))

array([[2, 3, 4, 0, 1],
       [3, 0, 0, 1, 1]])

array([[0.88079708, 0.95257413, 0.98201379, 0.11920292, 0.73105858],
       [0.95257413, 0.01798621, 0.5       , 0.73105858, 0.73105858]])

array([[ 0.96402758,  0.99505475,  0.9993293 , -0.96402758,  0.76159416],
       [ 0.99505475, -0.9993293 ,  0.        ,  0.76159416,  0.76159416]])

### Algorithms

#### Layer Dense

In [ ]:
class Dense2:
  class _jitted:  # jitted functions
    @staticmethod
    @njit
    def _initialize(n, input_dim, seed, activation):
      # return randomly generated Weight
      if seed is not None:
        np.random.seed(seed)

      if activation == 'relu':
        # 'HE' initialization  (He, 2015)
        return np.random.normal(0.0, np.sqrt(n), size=(n, input_dim))

      # elif activation == 'sigmoid' or activation == 'tanh':
        # 'Xavier' initialization  (Yoshua dan Xavier, 2010)
      tmp = 1/np.sqrt(n)
      return np.random.rand(n, input_dim) * tmp - tmp

    @njit
    def _count_Z(Weight, X, bias_):
      return np.dot(Weight, X) + bias_ 


  def __init__(self, n, input_dim=None, activation='relu',
               weights=None, bias=None, seed=None):
    
    self.rng = np.random.default_rng(seed)
    self.seed = seed

    self.activation = activation
    self.A_prev = None
    self.n = n

    self.Weight = weights
    self.bias_ = bias   # vektor bias

    if input_dim is not None:
      self.initialize(input_dim)

  def set_seed(self, seed):
    self.seed = seed
    self.rng = np.random.default_rng(seed)
    
  def initialize(self, input_dim):
    if self.Weight is None:
      self.Weight = self._jitted._initialize(self.n, input_dim, self.seed, 
                                             self.activation)

    if self.bias_ is None:
      self.bias_ = self.rng.random((self.n, 1)) * 2 + 1    
    
  def activate(self, X):
    self.A_prev = X
    assert self.Weight.shape[1] == X.shape[0], 'Incompatible shape'
    self.Z = self._jitted._count_Z(self.Weight, X, self.bias_)
    if self.activation == 'relu':
      return relu(self.Z)
    if self.activation == 'tanh':
      return tanh(self.Z)
    return sigmoid(self.Z)

  @property
  def initialized(self):
    return self.Weight is not None

                    
X = np.random.rand(10, 3)
layer = Dense2(5, 3, activation='sigmoid', 
               weights=np.arange(5 * 3, dtype='float64').reshape(5, 3))
layer.activate(np.ascontiguousarray(X.T))

array([[0.95871046, 0.91257544, 0.95643726, 0.95432599, 0.96798919,
        0.91308006, 0.86571591, 0.93988738, 0.96906541, 0.96097486],
       [0.99997061, 0.99628646, 0.99996116, 0.99991968, 0.99997998,
        0.9996681 , 0.99883044, 0.9996263 , 0.99995051, 0.99950232],
       [0.99999995, 0.99962559, 0.99999991, 0.99999965, 0.99999997,
        0.99999701, 0.99997717, 0.99999436, 0.9999998 , 0.99998423],
       [1.        , 0.9999856 , 1.        , 1.        , 1.        ,
        0.99999999, 0.99999983, 0.99999997, 1.        , 0.99999981],
       [1.        , 0.99999932, 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ]])

#### Neural Network

In [ ]:
class Nyunet2:
  def __init__(self, layers, seed=None):
    self.seed = seed
    self.layers = layers
    self._initialized = False
    
  def initialize(self, input_dim=None):
    dim = input_dim
    if dim is None and not self.layers[0].initialized:
      dim = self.layers[0].n
    assert dim is not None, 'Please provide input_dim'

    for i, layer in enumerate(self.layers):
      if self.seed is not None or not layer.initialized:
        if self.seed is not None:
          layer.set_seed(self.seed)
        layer.initialize(dim)
      dim = layer.n
    
    self._initialized = True

  def feedforward(self, X:nb.double[:, ::1]):
    if not self._initialized:
      self.initialize(X.shape[1])
    A = X.T
    for layer in self.layers:
      A = layer.activate(A)
    return A

  def predict(self, X):
    return self.feedforward(X).reshape((-1))

nn = Nyunet2([Dense2(5),
              Dense2(1)], 
             seed=10)
nn.predict(X)

array([5.30893172, 2.31652473, 6.97010183, 3.73921997, 4.04178183,
       4.95424709, 5.02271749, 7.30627184, 2.36764387, 0.49534614])

#### Particle Swarm

In [ ]:
class ParticleSwarm:
  class _jitted:
    @staticmethod
    @njit
    def _init(seed, n_particles, dim, ranges):
      if seed is not None:
        np.random.seed(seed)
      dist = ranges[1] - ranges[0]
      X = np.random.rand(n_particles, dim) * dist + ranges[0]   # matriks posisi
      V = np.random.rand(n_particles, dim)   # matriks kecepatan
      return (X, V)

    @njit
    def _step(fitness, X, p_fit_, P, V, v_max, phi, 
              _find_best_particle, verbosity=0):
      # update P partikel terbaik dan p_fitness terbaik
      mask = fitness > p_fit_
      P[mask] = X[mask]
      p_fit_ = np.maximum(fitness, p_fit_)
            
      # fitness terbaik secara global
      best_fit_index, best_particle = _find_best_particle(p_fit_, P)
      best_fit = p_fit_[best_fit_index]
      
      # update vektor kecepatan tiap partikel
      r = np.random.rand(1)
      V = V + r*(phi[0]*(P - X) + phi[1]*(best_particle - X))

      if v_max is not None:
        V = clip_np(V, a_min=-v_max, a_max=v_max)

      if verbosity > 1:
        print('fi', fitness.flatten())
        print('fp', (fitness == p_fit_).flatten())
        print('X ', X.flatten())

      # update vektor posisi tiap partikel
      X = X + V
      
      if verbosity > 1:
        print('P ', P.flatten())
        print('b ', best_particle, 'r', r)
        print('-0', (P-X).flatten())
        print('-1', (best_particle-X).flatten())
        print('V ', V.flatten())
        print('X ', X.flatten())

      return (X, p_fit_, P, V, v_max)

    @njit
    def _find_best_particle(p_fit_, P):
      best_fitness_index = np.argmax(p_fit_)
      best_particle = P[best_fitness_index]
      return (best_fitness_index, best_particle)

  def __init__(self, dim, 
               n_particles=30,    # rekomendasi (Carlisle dan Dozier, 2001)
               v_max:nb.float32=None, 
               # nilai-nilai phi atas rekomendasi (Carlisle dan Dozier, 2001)
               phi_1=2.8, phi_2=1.3,    
               ranges=(-1, 1),
               seed=None):
    self.rng = np.random.default_rng(seed)

    # self.X matriks posisi
    # self.V matriks kecepatan
    # self.p_fit_ vektor fitness terbaik tiap partikel 
    self.X, self.V = self._jitted._init(seed, n_particles, dim, ranges)
    self.p_fit_ = np.ones((n_particles,)) * float('-inf')

    self.v_max = v_max    # skalar kecepatan maksimal
    self.P = self.X   # matriks posisi solusi terbaik tiap partikel
    # self._best_fit = float('-inf')    # skalar fitness global terbaik
    self.phi = (phi_1, phi_2)   # learning rate
  
  def step(self, fitness_func, *args, verbosity=0):
    fitness = fitness_func(self.X, *args)
    self.X, self.p_fit_, self.P, self.V, \
    self.v_max = self._jitted._step(fitness, self.X, self.p_fit_, self.P, 
                                    self.V, self.v_max, self.phi, 
                                    self._jitted._find_best_particle, 
                                    verbosity)

  @property
  def best_particle(self):
    i, best_p = self._jitted._find_best_particle(self.p_fit_, self.P)
    return best_p

  @property
  def best_fitness(self):
    return np.max(self.p_fit_)

@njit
def fit_fn1(Particles, a=0, b=0, *args):
  return -((a - Particles[:, 0])**2 + (b - Particles[:, 1])**2)

@njit
def fit_fn2(Particles, x=0, *args):
  # X must be five
  return - np.abs(x - Particles[:, 0])

In [ ]:
swarm = ParticleSwarm(2, 3, v_max=1, ranges=(-6, 50), seed=1)
max_it = 30

swarm.step(fit_fn1, 100, 7.8)
ts = time.time()
for i in range(2000):
  swarm.step(fit_fn1, 100, 7.8, verbosity=0)
  # print(swarm.best_fitness)
  # print()

print(f'ETA : {time.time() - ts} s')
print(f'fit : {1+swarm.best_fitness}')
print(f'best: {swarm.best_particle}')

ETA : 0.03964853286743164 s
fit : 0.9999999189369814
best: [99.99999822  7.79971529]


#### Neural Net Builder Helper Function

In [ ]:
def calc_dim(arch, inp_dim):
  arc = [inp_dim, *[a[0] for a in arch]]
  return np.sum([arc[i-1] * arc[i] + 1 for i in range(1, len(arc))])

In [ ]:
def make_nn_layers(arch, x_, input_dim=None):
  layers = []
  ct = 0
  prev = input_dim
  for i, archi in enumerate(arch):
    arc, act = archi
    stop_w = ct + prev*arc 
    weights = x_[ct:stop_w].reshape(arc, prev)
    bias = x_[stop_w]
    if i == 0:
      layers.append(Dense2(arc, 
                           input_dim, 
                           weights=weights,
                           bias=bias,
                           activation=act))
    else:
      layers.append(Dense2(arc, 
                           weights=weights,
                           bias=bias,
                           activation=act))
    ct = stop_w + 1
    prev = arc
  return layers

nn_arch = ((3, 'relu'), (5, 'relu'), (2, 'tanh'), (1, 'sigmoid'))
dim = calc_dim(nn_arch, 2)
layers = make_nn_layers(nn_arch, np.arange(dim), 2)
for layer in layers:
  print(layer.n, layer.activation, 
        layer.Weight.shape, 
        layer.Weight.flatten(), 
        layer.bias_)

3 relu (3, 2) [0 1 2 3 4 5] 6
5 relu (5, 3) [ 7  8  9 10 11 12 13 14 15 16 17 18 19 20 21] 22
2 tanh (2, 5) [23 24 25 26 27 28 29 30 31 32] 33
1 sigmoid (1, 2) [34 35] 36


In [ ]:
@njit
def accuracy_of(y, y_pred):
  return (1 - np.abs(y - y_pred)).sum() / y.shape[0]

def train_nn_fit(Particles, X, y, arch):
  fitness = []
  for i in range(Particles.shape[0]):
    nn = Nyunet2(make_nn_layers(arch, Particles[i], X.shape[1]), 
                 seed=0)
    y_pred = nn.predict(X)
    score = accuracy_of(y, y_pred)
    # score = -np.sum((y - y_pred)**2)/y.shape[0]  # MSE
    # score = -np.sum((y - y_pred)**2)/y.shape[0] / np.var(y)  # rMSE (relative)
    # score = - np.sqrt(np.sum((y - y_pred)**2)/y.shape[0])   # RMSE (relative)
    fitness.append(score)
  return np.asarray(fitness).flatten()

accuracy_of(np.array([1, 0, 0]), np.array([1, 0, 1]));

In [ ]:
# Mengubah partikel weight menjadi model NN

def make_model(weight_particle, architecture, input_dim, seed=None):
  layers = make_nn_layers(architecture, weight_particle, input_dim)
  return Nyunet2(layers, seed)

### Testing

In [ ]:
def ground_truth(x1, x2):
  return x1 + x2 > 1

m = 100

rng = np.random.default_rng(2)
X = rng.random((m, 2)) * 10 - 5
y = ground_truth(X[:, 0], X[:, 1])
X_train, X_val, X_test = X[:60], X[60:80], X[80:]
y_train, y_val, y_test = y[:60], y[60:80], y[80:]
print(f'class balanceness: {y_train.sum() / y_train.shape[0]}')
print(f'class balanceness: {y_val.sum() / y_val.shape[0]}')
print(f'class balanceness: {y_test.sum() / y_test.shape[0]}')

class balanceness: 0.4666666666666667
class balanceness: 0.45
class balanceness: 0.5


In [ ]:
nn_arch = ((10, 'relu'), (20, 'relu'), (1, 'sigmoid'))

swarm = ParticleSwarm(calc_dim(nn_arch, 2), 
                      n_particles=50, 
                      ranges=(-1, 1),
                      seed=10)

# stop_ct = 0
# max_ct = 3
# prev = float('-inf')
best_fitness_hist = []

ts = time.time()
for i in range(10):
  swarm.step(train_nn_fit, X_train, y_train, nn_arch)
  if i % 1 == 0:
    model = make_model(swarm.best_particle, nn_arch, 2, 10)
    y_pred = model.predict(X_val)
    best_fitness_hist.append(accuracy_of(y_val, y_pred))
  # stop_ct =  stop_ct+1 if best_fit == prev else 0
  # if stop_ct >= max_ct-1:
  #   break
  # prev = best_fit
print(f'ETA: {time.time() - ts} s\n')
for i, fit in enumerate(best_fitness_hist):
  print(f'#{i} validation score: {fit}')

model = make_model(swarm.best_particle, nn_arch, 2, 10)
y_pred = model.predict(X_test)
print('test score:', accuracy_of(y_test, y_pred))

ETA: 2.998675584793091 s

#0 validation score: 0.8322808706045212
#1 validation score: 0.8322808706045212
#2 validation score: 0.8322808706045212
#3 validation score: 0.8322808706045212
#4 validation score: 0.8533927237929373
#5 validation score: 0.8533927237929373
#6 validation score: 0.8533927237929373
#7 validation score: 0.8533927237929373
#8 validation score: 0.8533927237929373
#9 validation score: 0.8533927237929373
test score: 0.9424813146463729


## Setting Up

In [ ]:
df_train = pd.read_csv('salju/salju_train.csv')
df_test = pd.read_csv('salju/salju_test.csv')
drop_target = df_train['BersaljuBesok'].isna()
df_train = df_train[~drop_target].reset_index(drop=True)

In [ ]:
# Buat model Pipeline
pl = Pipeline(pipes=[])

In [ ]:
pl.set_origin(df_train)
pl.peek().head()

,id,Tanggal,KodeLokasi,SuhuMin,SuhuMax,Hujan,Penguapan,SinarMatahari,ArahAnginTerkencang,KecepatanAnginTerkencang,ArahAngin9am,ArahAngin3pm,KecepatanAngin9am,KecepatanAngin3pm,Kelembaban9am,Kelembaban3pm,Tekanan9am,Tekanan3pm,Awan9am,Awan3pm,Suhu9am,Suhu3pm,BersaljuHariIni,BersaljuBesok
0,1,01/06/2014,C4,10.4,15.5,4.8,NaN,NaN,WSW,24.0,NaN,WSW,0.0,13.0,78.0,76.0,1020.1,1018.5,NaN,NaN,13.1,15.0,Ya,Tidak
1,2,15/07/2014,C10,9.0,17.0,8.0,2.6,7.4,NaN,NaN,SW,WNW,13.0,20.0,80.0,61.0,1015.2,1014.6,7.0,5.0,11.9,15.5,Ya,Ya
2,3,16/02/2011,C46,18.2,32.0,0.0,NaN,NaN,ESE,44.0,SE,SE,15.0,26.0,62.0,42.0,NaN,NaN,NaN,NaN,23.8,29.6,Tidak,Tidak
3,4,08/08/2012,C36,7.3,24.5,0.0,8.4,10.4,SSW,54.0,N,SW,13.0,19.0,25.0,17.0,1019.2,1016.9,1.0,7.0,15.3,23.2,Tidak,Tidak
4,5,29/10/2016,C7,5.9,20.3,0.0,3.6,12.6,N,37.0,NNW,ESE,22.0,19.0,55.0,48.0,1019.7,1014.7,2.0,6.0,12.4,18.1,Tidak,Tidak


## Data Preparation

In [ ]:
# Drop kolom id
order = 0
@pl.add_pipe(order, ('id'), skipable=True)
def drop_columns(df, args, _):
  return df.drop(args, axis=1)

In [ ]:
# Mengubah label 'Yes' menjadi 'Ya' dan label 'No' menjadi 'Tidak' 
# pada fitur 'Bersalju'
id = 1
@pl.add_pipe(id, skipable=False)
def handle_inconsistent(df, args, stash):
  return replace_values(df, ['BersaljuBesok', 'BersaljuHariIni'], 
            (('Yes', 'Ya'), ('No', 'Tidak')))

In [ ]:
import datetime as dt

In [ ]:
# Ubah kolom tanggal menjadi timestamp
id = 2
@pl.add_pipe(id)
def transform_tanggal(df, args, _):
  # return df['Tanggal']
  # df['Tanggal'] = pd.to_datetime(df['Tanggal'])
  df['Tanggal'] = (pd.to_datetime(df['Tanggal']) - dt.datetime(1970,1,1)).dt.total_seconds()
  return df

pl.peek(id)

Executing[0]: drop_columns...
took 0.0161 sec
Executing[1]: handle_inconsistent...
took 0.0370 sec
Executing[2]: transform_tanggal...
took 0.0912 sec


,Tanggal,KodeLokasi,SuhuMin,SuhuMax,Hujan,Penguapan,SinarMatahari,ArahAnginTerkencang,KecepatanAnginTerkencang,ArahAngin9am,ArahAngin3pm,KecepatanAngin9am,KecepatanAngin3pm,Kelembaban9am,Kelembaban3pm,Tekanan9am,Tekanan3pm,Awan9am,Awan3pm,Suhu9am,Suhu3pm,BersaljuHariIni,BersaljuBesok
0,1.388966e+09,C4,10.4,15.5,4.8,NaN,NaN,WSW,24.0,NaN,WSW,0.0,13.0,78.0,76.0,1020.1,1018.5,NaN,NaN,13.1,15.0,Ya,Tidak
1,1.405382e+09,C10,9.0,17.0,8.0,2.6,7.4,NaN,NaN,SW,WNW,13.0,20.0,80.0,61.0,1015.2,1014.6,7.0,5.0,11.9,15.5,Ya,Ya
2,1.297814e+09,C46,18.2,32.0,0.0,NaN,NaN,ESE,44.0,SE,SE,15.0,26.0,62.0,42.0,NaN,NaN,NaN,NaN,23.8,29.6,Tidak,Tidak
3,1.344384e+09,C36,7.3,24.5,0.0,8.4,10.4,SSW,54.0,N,SW,13.0,19.0,25.0,17.0,1019.2,1016.9,1.0,7.0,15.3,23.2,Tidak,Tidak
4,1.477699e+09,C7,5.9,20.3,0.0,3.6,12.6,N,37.0,NNW,ESE,22.0,19.0,55.0,48.0,1019.7,1014.7,2.0,6.0,12.4,18.1,Tidak,Tidak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106659,1.233360e+09,C38,20.1,23.7,0.0,7.2,8.9,ESE,43.0,SE,ESE,24.0,26.0,74.0,70.0,1019.3,1017.6,4.0,6.0,22.0,22.1,Tidak,Ya
106660,1.268266e+09,C16,15.7,25.2,0.0,NaN,NaN,SSE,37.0,SSE,E,28.0,19.0,52.0,52.0,1018.4,1016.2,NaN,NaN,23.8,24.4,Tidak,Ya
106661,1.289434e+09,C17,7.5,20.4,1.6,NaN,NaN,NW,33.0,N,NW,4.0,13.0,92.0,51.0,1014.5,1010.6,NaN,NaN,12.6,19.8,Ya,Tidak
106662,1.334534e+09,C11,10.8,29.8,0.0,7.8,11.2,E,48.0,ESE,SE,13.0,26.0,35.0,18.0,1020.0,1015.8,0.0,1.0,21.7,29.2,Tidak,Tidak


In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Mengecek apakah semua KodeLokasi terdapat pada data train
(df_train['KodeLokasi'].sort_values().unique() == df_test['KodeLokasi'].sort_values().unique()).all()

True

In [ ]:
# Mengecek apakah semua ArahAngin terdapat pada data train
(df_train.loc[df_train['ArahAngin9am'].notna(), 'ArahAngin9am'].sort_values().unique() \
== df_test.loc[df_test['ArahAngin9am'].notna(), 'ArahAngin9am'].sort_values().unique()).all()

True

In [ ]:
# Mengecek apakah ada tanggal yang hilang di kedua dataset
df_train['Tanggal'].isna().sum() + df_test['Tanggal'].isna().sum()

0

In [ ]:
def columns_like(df, col_name):
  return (col for col in df.columns if col_name in col)

In [ ]:
class BatchLabelEncoder(LabelEncoder):
  def fit(self, label_dict):
    self._labels = label_dict
    for key in self._labels.keys():
      self._labels[key] = LabelEncoder().fit(self._labels[key])
    return self
  
  def transform(self, df_input):
    df = df_input.loc[:]
    for key in self._labels.keys():
      for col in columns_like(df, key):
        # hanya proses yang bukan NaN
        df.loc[df[col].notna(), col] = self._labels[key].transform(df.loc[df[col].notna(), col])
    return df

  def inverse_transform(self, _df):
    df = _df.loc[:]
    for key in self._labels.keys():
      for col in columns_like(df, key):
        # df[col] = df[col].round()
        df.loc[df[col].notna(), col] = self._labels[key].inverse_transform(df.loc[df[col].notna(), col].astype('int64'))
    return df


# test suite
ble = BatchLabelEncoder().fit({
      'KodeLokasi': [f'C{i}' for i in range(50)],
      'ArahAngin': ['E', 'ENE', 'ESE', 'N', 'NE', 'NNE', 'NNW', 'NW', 'S', 
                    'SE', 'SSE', 'SSW', 'SW', 'W', 'WNW', 'WSW'],
      'Bersalju': ['Tidak', 'Ya']
  })
df = pl.peek(2, verbose=0)
df_encoded = ble.transform(df)
df_encoded['KodeLokasi'] += 0.9  # uji bilangan float
df_inversed = ble.inverse_transform(df_encoded)
mask = df == df_inversed

assert (mask == df.notna()).all().all()  # semua nilai hasil inverse sama dengan awal

In [ ]:
id = 3
@pl.add_pipe(id)
def label_encode(df, args, stash):
  labels = {
      'KodeLokasi': [f'C{i}' for i in range(50)],
      'ArahAngin': ['E', 'ENE', 'ESE', 'N', 'NE', 'NNE', 'NNW', 'NW', 'S', 
                    'SE', 'SSE', 'SSW', 'SW', 'W', 'WNW', 'WSW'],
      'Bersalju': ['Tidak', 'Ya']
  }
  ble = BatchLabelEncoder().fit(labels)
  stash['ble'] = ble  # tambahkan model le kedalam stash
  return ble.transform(df)

pl.peek(id, verbose=0).head()

,Tanggal,KodeLokasi,SuhuMin,SuhuMax,Hujan,Penguapan,SinarMatahari,ArahAnginTerkencang,KecepatanAnginTerkencang,ArahAngin9am,ArahAngin3pm,KecepatanAngin9am,KecepatanAngin3pm,Kelembaban9am,Kelembaban3pm,Tekanan9am,Tekanan3pm,Awan9am,Awan3pm,Suhu9am,Suhu3pm,BersaljuHariIni,BersaljuBesok
0,1.388966e+09,34,10.4,15.5,4.8,NaN,NaN,15,24.0,NaN,15,0.0,13.0,78.0,76.0,1020.1,1018.5,NaN,NaN,13.1,15.0,1,0
1,1.405382e+09,2,9.0,17.0,8.0,2.6,7.4,NaN,NaN,12,14,13.0,20.0,80.0,61.0,1015.2,1014.6,7.0,5.0,11.9,15.5,1,1
2,1.297814e+09,41,18.2,32.0,0.0,NaN,NaN,2,44.0,9,9,15.0,26.0,62.0,42.0,NaN,NaN,NaN,NaN,23.8,29.6,0,0
3,1.344384e+09,30,7.3,24.5,0.0,8.4,10.4,11,54.0,3,12,13.0,19.0,25.0,17.0,1019.2,1016.9,1.0,7.0,15.3,23.2,0,0
4,1.477699e+09,47,5.9,20.3,0.0,3.6,12.6,3,37.0,6,2,22.0,19.0,55.0,48.0,1019.7,1014.7,2.0,6.0,12.4,18.1,0,0


In [ ]:
# Membangun model imputer atau menggunakan yang sudah ada
df = pl.peek(3, verbose=0)

if not USE_PREPROCESSED:
    df = df.drop('Tanggal', axis=1)
    imputer = KNNImputer(n_neighbors=10).fit(df)

pl.add_stash('imputer', imputer);

In [ ]:
# Mengisi missing values dengan algoritma KNN Imputer
id = 4
@pl.add_pipe(id)
## @jit(parallel=True, fastmath=True)
def knn_impute(df, args, stash):
  imputer = stash['imputer']
  df_tmp = df.drop('Tanggal', axis=1)
  imputed = imputer.transform(df_tmp)
  df[df_tmp.columns] = imputed
  return df

In [ ]:
# Untuk skip imputing
if not USE_PREPROCESSED:
    df = pl.peek(id, cache=True)  # disimpan di cache supaya tidak perlu run lagi
    df.to_csv('salju/salju_train_imputed.csv', index=None)
    !zip sti.zip salju/salju_train_imputed.csv
    files.download('sti.zip')
else:
    # Skip imputing
    df = pd.read_csv('salju/salju_train_imputed.csv')
    pl.set_cache(df, id)

In [ ]:
id = 5
@pl.add_pipe(id, ['BersaljuBesok'])
def stash_y(df, feat, stash):
  stash['y'] = df.loc[:, feat].to_numpy()
  return df.drop(feat, axis=1)

In [ ]:
# Melakukan inverse dari label encoding agar memperoleh kembali kolom kategori
id = 6
@pl.add_pipe(id)
def label_decode(df, args, stash):
  ble = stash['ble']
  df_tmp = ble.inverse_transform(df.drop('Tanggal', axis=1))
  return pd.concat((df['Tanggal'], df_tmp), axis=1)

In [ ]:
# Melakukan one-hot-encoding terhadap fitur-fitur berikut
df = pl.peek(6)
categorical_features = [*columns_like(df, 'BersaljuHariIni'), 
                        *columns_like(df, 'ArahAngin'), 
                        'KodeLokasi']
id = 7
@pl.add_pipe(id, categorical_features)
def one_hot(df, features, _):
  return one_hot_encode(df, features)

pl.peek(id, verbose=0)

Executing[5]: stash_y...
took 0.0213 sec
Executing[6]: label_decode...
took 0.3528 sec


,Tanggal,SuhuMin,SuhuMax,Hujan,Penguapan,SinarMatahari,KecepatanAnginTerkencang,KecepatanAngin9am,KecepatanAngin3pm,Kelembaban9am,Kelembaban3pm,Tekanan9am,Tekanan3pm,Awan9am,Awan3pm,Suhu9am,Suhu3pm,BersaljuHariIni_Tidak,BersaljuHariIni_Ya,ArahAnginTerkencang_E,ArahAnginTerkencang_ENE,ArahAnginTerkencang_ESE,ArahAnginTerkencang_N,ArahAnginTerkencang_NE,ArahAnginTerkencang_NNE,ArahAnginTerkencang_NNW,ArahAnginTerkencang_NW,ArahAnginTerkencang_S,ArahAnginTerkencang_SE,ArahAnginTerkencang_SSE,ArahAnginTerkencang_SSW,ArahAnginTerkencang_SW,ArahAnginTerkencang_W,ArahAnginTerkencang_WNW,ArahAnginTerkencang_WSW,ArahAngin9am_E,ArahAngin9am_ENE,ArahAngin9am_ESE,ArahAngin9am_N,ArahAngin9am_NE,...,KodeLokasi_C18,KodeLokasi_C19,KodeLokasi_C2,KodeLokasi_C20,KodeLokasi_C21,KodeLokasi_C22,KodeLokasi_C23,KodeLokasi_C24,KodeLokasi_C25,KodeLokasi_C26,KodeLokasi_C27,KodeLokasi_C28,KodeLokasi_C29,KodeLokasi_C3,KodeLokasi_C30,KodeLokasi_C31,KodeLokasi_C32,KodeLokasi_C33,KodeLokasi_C34,KodeLokasi_C35,KodeLokasi_C36,KodeLokasi_C37,KodeLokasi_C38,KodeLokasi_C39,KodeLokasi_C4,KodeLokasi_C40,KodeLokasi_C41,KodeLokasi_C42,KodeLokasi_C43,KodeLokasi_C44,KodeLokasi_C45,KodeLokasi_C46,KodeLokasi_C47,KodeLokasi_C48,KodeLokasi_C49,KodeLokasi_C5,KodeLokasi_C6,KodeLokasi_C7,KodeLokasi_C8,KodeLokasi_C9
0,1.388966e+09,10.4,15.5,4.8,2.17,4.64,24.0,0.0,13.0,78.0,76.0,1020.10,1018.50,4.7,4.4,13.1,15.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.405382e+09,9.0,17.0,8.0,2.60,7.40,38.0,13.0,20.0,80.0,61.0,1015.20,1014.60,7.0,5.0,11.9,15.5,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.297814e+09,18.2,32.0,0.0,5.80,9.99,44.0,15.0,26.0,62.0,42.0,1014.07,1010.97,2.5,3.7,23.8,29.6,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,1.344384e+09,7.3,24.5,0.0,8.40,10.40,54.0,13.0,19.0,25.0,17.0,1019.20,1016.90,1.0,7.0,15.3,23.2,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.477699e+09,5.9,20.3,0.0,3.60,12.60,37.0,22.0,19.0,55.0,48.0,1019.70,1014.70,2.0,6.0,12.4,18.1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106659,1.233360e+09,20.1,23.7,0.0,7.20,8.90,43.0,24.0,26.0,74.0,70.0,1019.30,1017.60,4.0,6.0,22.0,22.1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
106660,1.268266e+09,15.7,25.2,0.0,6.78,7.24,37.0,28.0,19.0,52.0,52.0,1018.40,1016.20,3.5,3.9,23.8,24.4,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
106661,1.289434e+09,7.5,20.4,1.6,3.96,5.01,33.0,4.0,13.0,92.0,51.0,1014.50,1010.60,1.9,4.7,12.6,19.8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
106662,1.334534e+09,10.8,29.8,0.0,7.80,11.20,48.0,13.0,26.0,35.0,18.0,1020.00,1015.80,0.0,1.0,21.7,29.2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from scipy import stats

In [ ]:
df = pl.peek(id, verbose=0)
categorical_features = [*columns_like(df, 'BersaljuHariIni'), 
                        *columns_like(df, 'ArahAngin'), 
                        *columns_like(df, 'KodeLokasi')]
numerical_features = df.drop(categorical_features, axis=1).columns

In [ ]:
# Melakukan standardisasi data untuk fitur-fitur numerik
id = 8
@pl.add_pipe(id, numerical_features)
def standardize(df, columns, stash):
  df = df.loc[:]
  df[columns] = stats.zscore(df[columns])
  return df

pl.peek(id, verbose=0).describe()

,Tanggal,SuhuMin,SuhuMax,Hujan,Penguapan,SinarMatahari,KecepatanAnginTerkencang,KecepatanAngin9am,KecepatanAngin3pm,Kelembaban9am,Kelembaban3pm,Tekanan9am,Tekanan3pm,Awan9am,Awan3pm,Suhu9am,Suhu3pm,BersaljuHariIni_Tidak,BersaljuHariIni_Ya,ArahAnginTerkencang_E,ArahAnginTerkencang_ENE,ArahAnginTerkencang_ESE,ArahAnginTerkencang_N,ArahAnginTerkencang_NE,ArahAnginTerkencang_NNE,ArahAnginTerkencang_NNW,ArahAnginTerkencang_NW,ArahAnginTerkencang_S,ArahAnginTerkencang_SE,ArahAnginTerkencang_SSE,ArahAnginTerkencang_SSW,ArahAnginTerkencang_SW,ArahAnginTerkencang_W,ArahAnginTerkencang_WNW,ArahAnginTerkencang_WSW,ArahAngin9am_E,ArahAngin9am_ENE,ArahAngin9am_ESE,ArahAngin9am_N,ArahAngin9am_NE,...,KodeLokasi_C18,KodeLokasi_C19,KodeLokasi_C2,KodeLokasi_C20,KodeLokasi_C21,KodeLokasi_C22,KodeLokasi_C23,KodeLokasi_C24,KodeLokasi_C25,KodeLokasi_C26,KodeLokasi_C27,KodeLokasi_C28,KodeLokasi_C29,KodeLokasi_C3,KodeLokasi_C30,KodeLokasi_C31,KodeLokasi_C32,KodeLokasi_C33,KodeLokasi_C34,KodeLokasi_C35,KodeLokasi_C36,KodeLokasi_C37,KodeLokasi_C38,KodeLokasi_C39,KodeLokasi_C4,KodeLokasi_C40,KodeLokasi_C41,KodeLokasi_C42,KodeLokasi_C43,KodeLokasi_C44,KodeLokasi_C45,KodeLokasi_C46,KodeLokasi_C47,KodeLokasi_C48,KodeLokasi_C49,KodeLokasi_C5,KodeLokasi_C6,KodeLokasi_C7,KodeLokasi_C8,KodeLokasi_C9
count,1.066640e+05,1.066640e+05,1.066640e+05,1.066640e+05,1.066640e+05,1.066640e+05,1.066640e+05,1.066640e+05,1.066640e+05,1.066640e+05,1.066640e+05,1.066640e+05,1.066640e+05,1.066640e+05,1.066640e+05,1.066640e+05,1.066640e+05,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,...,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.00000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000,106664.000000
mean,-1.394846e-16,1.684569e-16,-5.429230e-17,2.887198e-15,2.825103e-17,2.660269e-16,7.925645e-16,-9.306799e-16,1.099083e-15,2.478782e-17,1.564767e-16,-6.033208e-15,-3.268505e-15,-8.633935e-17,-6.772148e-17,-1.597023e-16,-4.083919e-17,0.777966,0.222034,0.064108,0.057011,0.051526,0.065936,0.054339,0.053589,0.056880,0.067792,0.073211,0.073117,0.067464,0.062533,0.062739,0.069995,0.056898,0.062861,0.063508,0.054770,0.053429,0.082699,0.058633,...,0.021516,0.020363,0.023682,0.021432,0.020879,0.020879,0.021301,0.020729,0.021451,0.020841,0.020804,0.021179,0.020776,0.022444,0.020260,0.021038,0.020879,0.020691,0.021010,0.021441,0.020907,0.020288,0.021497,0.01920,0.021497,0.021113,0.018057,0.020635,0.020654,0.019604,0.020916,0.020916,0.011110,0.010969,0.010894,0.017213,0.022219,0.022866,0.022688,0.020907
std,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,0.415616,0.415616,0.244946,0.231864,0.221070,0.248171,0.226686,0.225206,0.231613,0.251390,0.260484,0.260330,0.250825,0.242122,0.242494,0.255141,0.231649,0.242714,0.243875,0.227532,0.224889,0.275428,0.234937,...,0.145098,0.141239,0.152057,0.144819,0.142979,0.142979,0.144385,0.142475,0.144881,0.142853,0.142727,0.143980,0.142633,0.148124,0.140889,0.143512,0.142979,0.142349,0.143418,0.144850,0.143073,0.140984,0.145036,0.13723,0.145036,0.143762,0.133157,0.142159,0.142223,0.138634,0.143104,0.143104,0.104816,0.104158,0.103805,0.130065,0.147397,0.149478,0.148908,0.143073
m

In [ ]:
from numba import jit, njit  # untuk efisiensi digunakan parallel computing

In [ ]:
@njit
def expand_one_hot(arr):
  return arr * 2 - 1

# Dijalankan supaya fungsi di atas ter-compile terlebih dahulu. 
display(expand_one_hot(np.array([1, 0, 0, 0, 1, 1, 1, 0])))
display(expand_one_hot(np.array([[1, 0, 0, 0, 1, 1, 1, 0],
                                 [0, 0, 0, 0, 0, 1, 1, 1]])))

array([ 1, -1, -1, -1,  1,  1,  1, -1])

array([[ 1, -1, -1, -1,  1,  1,  1, -1],
       [-1, -1, -1, -1, -1,  1,  1,  1]])

In [ ]:
id = 9

@pl.add_pipe(id, categorical_features)
def min_one_one_encoding(df, features, _):
  arr = df[features].to_numpy()
  df = df.loc[:]
  df[features] = expand_one_hot(arr)
  return df

pl.peek(id)[categorical_features].head()

Executing[5]: stash_y...
took 0.0085 sec
Executing[6]: label_decode...
took 0.3158 sec
Executing[7]: one_hot...
took 0.3359 sec
Executing[8]: standardize...
took 0.3292 sec
Executing[9]: min_one_one_encoding...
took 3.0640 sec


,BersaljuHariIni_Tidak,BersaljuHariIni_Ya,ArahAnginTerkencang_E,ArahAnginTerkencang_ENE,ArahAnginTerkencang_ESE,ArahAnginTerkencang_N,ArahAnginTerkencang_NE,ArahAnginTerkencang_NNE,ArahAnginTerkencang_NNW,ArahAnginTerkencang_NW,ArahAnginTerkencang_S,ArahAnginTerkencang_SE,ArahAnginTerkencang_SSE,ArahAnginTerkencang_SSW,ArahAnginTerkencang_SW,ArahAnginTerkencang_W,ArahAnginTerkencang_WNW,ArahAnginTerkencang_WSW,ArahAngin9am_E,ArahAngin9am_ENE,ArahAngin9am_ESE,ArahAngin9am_N,ArahAngin9am_NE,ArahAngin9am_NNE,ArahAngin9am_NNW,ArahAngin9am_NW,ArahAngin9am_S,ArahAngin9am_SE,ArahAngin9am_SSE,ArahAngin9am_SSW,ArahAngin9am_SW,ArahAngin9am_W,ArahAngin9am_WNW,ArahAngin9am_WSW,ArahAngin3pm_E,ArahAngin3pm_ENE,ArahAngin3pm_ESE,ArahAngin3pm_N,ArahAngin3pm_NE,ArahAngin3pm_NNE,...,KodeLokasi_C18,KodeLokasi_C19,KodeLokasi_C2,KodeLokasi_C20,KodeLokasi_C21,KodeLokasi_C22,KodeLokasi_C23,KodeLokasi_C24,KodeLokasi_C25,KodeLokasi_C26,KodeLokasi_C27,KodeLokasi_C28,KodeLokasi_C29,KodeLokasi_C3,KodeLokasi_C30,KodeLokasi_C31,KodeLokasi_C32,KodeLokasi_C33,KodeLokasi_C34,KodeLokasi_C35,KodeLokasi_C36,KodeLokasi_C37,KodeLokasi_C38,KodeLokasi_C39,KodeLokasi_C4,KodeLokasi_C40,KodeLokasi_C41,KodeLokasi_C42,KodeLokasi_C43,KodeLokasi_C44,KodeLokasi_C45,KodeLokasi_C46,KodeLokasi_C47,KodeLokasi_C48,KodeLokasi_C49,KodeLokasi_C5,KodeLokasi_C6,KodeLokasi_C7,KodeLokasi_C8,KodeLokasi_C9
0,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1
3,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1


In [ ]:
if not USE_PREPROCESSED:
  df = pl.peek(verbose=0)
  df['y'] = pl._stash['y']

  msk = np.random.rand(len(df.index)) < 0.2
  df_val = df.loc[msk]
  df_train = df.loc[~msk]
  # df_train.to_csv('salju/df_train.csv', index=None)
  # df_val.to_csv('salju/df_val.csv', index=None)

  # !zip df_train_val.zip salju/df_train.csv salju/df_val.csv
  # files.download('df_train_val.zip')

In [ ]:
if not USE_PREPROCESSED:
  !rm df_train_val.zip

In [ ]:
def split_x_y(df):
  X = df.drop('y', axis=1).to_numpy()
  y = df['y'].to_numpy()
  return X, y

In [ ]:
if not USE_PREPROCESSED:
  X_train, y_train = split_x_y(df_train)
  X_val, y_val = split_x_y(df_val)
  with open('salju/salju_ds.pkl', 'wb') as f:
    pickle.dump({
        'X_train': X_train,
        'y_train': y_train,
        'X_val': X_val, 
        'y_val': y_val
    }, f)

  !zip salju_ds.zip salju/salju_ds.pkl
  files.download('salju_ds.zip')
else:
  with open('salju/salju_ds.pkl', 'rb') as f:
    dataset = pickle.load(f)
    X_train =  dataset['X_train']
    y_train =  dataset['y_train']
    X_val =  dataset['X_val']
    y_val =  dataset['y_val']

In [ ]:
if not USE_PREPROCESSED:
  !rm salju_ds.zip

## Train

In [ ]:
from jordiyapz_ai_toolkit.model.nyunet import NyuNet, Layer

### Manual Model

In [ ]:
model = NyuNet([
    Layer.Input(X_train.shape[1]),
    Layer.Linear(41, activation='relu'),
    Layer.Linear(3),
    Layer.Linear(1)
])

In [ ]:
model.fit(X_train, y_train.reshape((-1, 1)), epochs=10, learning_rate=.0005)

Epochs: 100%|██████████| 10/10 [00:03<00:00,  2.91it/s]


[0.7004161707211127,
 2.6231022674839206,
 0.6353468839288995,
 2.336916366965367,
 4.517614967578219,
 1.9034431383893988,
 0.6725799961783016,
 0.6003258425125444,
 0.6989442264835436,
 0.5925591901591949]

In [ ]:
model.evaluate(X_val, y_val, metrics=['accuracy'])

(array([0.77449658]),)

In [ ]:
from jordiyapz_ai_toolkit.method import Validation

In [ ]:
y_pred = np.round(model.feedforward(X_val.T))
y_pred = y_pred.reshape(y_val.shape)
conf_mat = Validation.confusion_matrix(y_val, y_pred)
conf_mat

Predicted,0.0,All
Actual,,
0.0,16654,16654
1.0,4849,4849
All,21503,21503


### Find Model with Auto-ML


In [ ]:
@njit
def scale_array(X, origin, target):
  ori_min, ori_max = origin
  targ_min, targ_max = target
  return (X - ori_min) \
    / (ori_max - ori_min) \
    * (targ_max - targ_min) \
    + targ_min

scale_array(np.random.rand(5, 2) * 10 - 5, (-5, 5), (2, 3))
scale_array(np.random.rand(5, 2) * 5, (0, 5), (0, 2))

array([[0.41160828, 0.81868895],
       [1.45711411, 1.17983798],
       [1.88239194, 0.89377477],
       [1.23516756, 0.16497647],
       [0.460826  , 0.24397338]])

In [ ]:
@njit
def decode_structure(Particles_s, max_range, num_of_activations): 
  Particles = clip_np(Particles_s, 0, max_range)
  max_layers = (Particles.shape[1] - 1) // 2
  layer_count_ = cast_to_int_1d(Particles[:, 0] * (max_layers) / max_range)+1
  layer_count_ = clip_np(layer_count_, 0, max_layers-1)
  Layers = cast_to_int_2d(Particles[:, 1:])
  N_each_layers = Layers[:, :max_layers] + 1
  Activation_enc = scale_array(Layers[:, max_layers:], (0, max_range), 
                               (0, num_of_activations-1))
  Activation_enc = cast_to_int_2d(Activation_enc)
  return (layer_count_, N_each_layers, Activation_enc)

def parse_architecture(layer_count, n_each_layers, 
                       act_each_layers, activations):
  
  try:
    architecture = [(n_each_layers[i], activations[act_each_layers[i]])
                    for i in range(layer_count)]
  except IndexError:
    print('Index error occured!', layer_count, 
          len(n_each_layers), 
          len(act_each_layers),
          len(activations))
    print(act_each_layers)
    architecture = [(n_each_layers[i], activations[act_each_layers[i]])
                    for i in range(min(len(n_each_layers), len(act_each_layers)))]
  
  # tambahkan juga layer output akhir
  architecture.append((1, 'sigmoid'))

  return tuple(architecture)

In [ ]:
def fitfn_structure_find(S_particles, best_model, X, y, X_val, y_val, max_range, 
                         activations=('relu',), seed=None, max_ct=3):
  '''
  Mencari struktur yang tepat
  
  enkoding: [jumlah layer, 
             n layer #1, n layer #2, ..., n layer #L, 
             arsi layer #1, arsi layer #2, ..., arsi layer #L]

  Struktur nn nya adalah sebagai berikut:
     input (bukan layer) - h layer #1 - h layer #2 - ... - output (sudah fix)
  
  Jadi yang direpresentasikan hanyalah jumlah hidden layer, banyaknya
  neuron, dan arsitektur masing-masing hidden layer tersebut.

  Keterangan:
    > len(enkoding) == dim, 
    > masing-masing n layer berada pada interval [ranges[0]+1, ranges[1]]
    > maksimal hidden layer == dimensi partikel - 1
  '''

  layer_count_, N_each_layers, Act_enc = decode_structure(S_particles, 
                                                          max_range, 
                                                          len(activations))
  
  best_scores = []

  for layer_count, n_each_layers, act_enc in zip(layer_count_, 
                                                 N_each_layers, 
                                                 Act_enc):
    
    architecture = parse_architecture(layer_count, n_each_layers, act_enc, 
                                      activations)
    
    swarm = ParticleSwarm(calc_dim(architecture, X.shape[1]), n_particles=20,
                          ranges=(-1, 1), seed=seed)
    
    stop_ct = 0
    # max_ct = 20
    prev_fitness = float('-inf')
    prev_model = None
    it = 0
    best_particle = None
    best_fitness = 0
    print(f'Trying {architecture}')
    while True:
      # if (it == 1): pdb.set_trace()
      swarm.step(train_nn_fit, X, y, architecture)
      best_fitness = swarm.best_fitness
      best_particle = swarm.best_particle

      if np.isnan(best_fitness):
        print('Skipping model due to nan value')
        best_particle = None
        best_fitness = 0
        break
      
      it += 1
      print(f'Iteration {it}: {best_fitness}')

      stop_ct = stop_ct+1 if best_fitness == prev_fitness else 0
      if stop_ct >= max_ct-1:
        break
      prev_fitness = best_fitness
    
    if best_particle is not None:
      weight_particle = (swarm.best_particle)
      model = make_model(weight_particle, architecture, X.shape[1], seed)
      y_pred = model.predict(X_val)
      score = accuracy_of(y_val, y_pred)
      best_scores.append(score)
      print(f'Score: {score}', end=' ')
      if score > best_model[0][0]:
        best_model[0] = (score, swarm.best_particle, architecture) 
        print('BEST!')
      else:
        print()
    else: 
      best_scores.append(0.)
    
  return np.asarray(best_scores)

In [ ]:
best_model = [(0, None, ())]

In [ ]:
max_range = 30
activations = ('relu', 'tanh')
max_num_of_layers = 2
num_of_particles = 5
seed = None  # 12

dim = 2*max_num_of_layers + 1

if not USE_PRETRAINED:
  swarm = ParticleSwarm(dim, num_of_particles, ranges=(0, max_range), seed=seed)
  # for i in range(5):
  i = 0
  while True:
    i += 1
    print(f'Generation {i}')
    max_ct = i*2 + 1
    print(f'Stopping Crit: max_no_improvement({max_ct})')
    swarm.step(fitfn_structure_find, best_model, X_train, y_train, X_val, y_val, 
              max_range, activations, seed, max_ct)
    best_fit = swarm.best_fitness
    print(f'Best Fitness: {best_fit}')
    print()
    if best_fit >= .9:
      break

In [ ]:
if not USE_PRETRAINED:
  score, weights, arch = best_model[0]
  model = Nyunet2(make_nn_layers(arch, weights, X_val.shape[1]))
  filename = f'model_{np.floor(score*10000).astype(int)}.pkl'
  with open(filename, 'wb') as f:
    pickle.dump((score, weights, arch), f)
  files.download(filename)
else:
  with open('salju/model.pkl', 'rb') as f:
    score, weights, arch = pickle.load(f)
    model = Nyunet2(make_nn_layers(arch, weights, X_train.shape[1]))
print(f'Best model architecture: {arch}')
print(f'Best model scored: {score}')

Best model architecture: ((41, 'relu'), (1, 'sigmoid'))
Best model scored: 0.817247644109855


## Test

In [ ]:
df_test = pd.read_csv('salju/salju_test.csv')

In [ ]:
if not USE_PREPROCESSED:
  pl.reset()
  try:
    pl.set_origin(df_test)
  except ValueError:
    # errornya cuma karena printing aja
    pass
  df = pl.compile(skip=True)

In [ ]:
if not USE_PREPROCESSED:
  df = pl.peek(verbose=0)
  df['y'] = pl._stash['y']

  df_ = df.loc[:]

In [ ]:
if not USE_PREPROCESSED:
  df_ = pl.peek()
  X_test, y_test = split_x_y(df_)

  with open('salju/salju_test.pkl', 'wb') as f:
    pickle.dump({
        'X_test': X_test,
        'y_test': y_test,
    }, f)

  !zip salju_test.zip salju/salju_test.pkl
  files.download('salju_test.zip')
else:
  with open('salju/salju_test.pkl', 'rb') as f:
    dataset = pickle.load(f)
    X_test =  dataset['X_test']
    y_test =  dataset['y_test']

In [ ]:
y_pred = model.predict(X_test)
accuracy_of(y_test, y_pred)

0.8180673193268053

## Referensi

1. [How To Standardize Data ...](https://visualstudiomagazine.com/articles/2014/01/01/how-to-standardize-data-for-neural-networks.aspx)
2. https://machinelearningmastery.com/weight-initialization-for-deep-learning-neural-networks/

## Deprecated

In [ ]:
#@title
# Menggantikan fungsi np.clip numpy karena tidak disupport oleh numba
# Deprekasi karena hanya menangani array 1 dimensi
@nb.extending.overload(np.clip)
def np_clip(a, a_min, a_max, out=None):
  def np_clip_impl(a, a_min, a_max, out=None):
    if out is None:
      out = np.empty_like(a)
    for i in range(len(a)):
      if a[i] < a_min:
        out[i] = a_min
      elif a[i] > a_max:
        out[i] = a_max
      else:
        out[i] = a[i]
    return out
  return np_clip_impl

In [ ]:
#@title
@jitclass([('n', nb.uint16),
           ('activation', nb.types.unicode_type),
          #  ('activation', nb.typeof(relu)),
           ('Weight', nb.optional(nb.double[:, ::1])),
           ('b_', nb.double[:, ::1]),
           ('Z', (nb.double[:, ::1])),
           ('A_prev', nb.optional(nb.double[:, ::1])),
           ])

class Dense_dep:
  def __init__(self, n, input_dim=None, activation='relu', 
               weights=None, bias=None, seed=None):
    if seed is not None:
      np.random.seed(seed)
    self.activation = activation #if activation=='relu' else None
    self.A_prev = None
    self.n = n

    self.Weight = weights
    self.b_ = bias if bias is not None else np.random.rand(n, 1)*2+1   # vektor bias

    if input_dim is not None:
      self.initialize(input_dim)
  
  def initialize(self, input_dim):
    if self.activation == 'relu':
      # 'HE' initialization  (He, 2015)
      self.Weight = np.random.normal(0.0, np.sqrt(self.n), size=(self.n, input_dim))
    else:
      # 'Xavier' initialization  (Yoshua dan Xavier, 2010)
      tmp = 1/np.sqrt(self.n)
      self.Weight = np.random.rand(self.n, input_dim) * tmp - tmp

  @property
  def initialized(self):
    return self.Weight is not None
  
  # def activation(self, Z):
  #   # return np.maximum(Z, 0)
  #   return relu(Z)

  def activate(self, X):
    self.A_prev = X
    self.Z = np.dot(self.Weight, X) + self.b_
    if self.activation == 'relu':
      return relu(self.Z)
    return sigmoid(self.Z)
    # return self.activation(self.Z)

layer = Dense_dep(5, 3, activation='sigmoid')
layer.activate(np.random.rand(3, 12));

In [ ]:
#@title
@jitclass([('layers', nb.types.ListType(Dense_dep.class_type.instance_type))])
class NyuNet2_dep(object):
  def __init__(self, layers):
    self.layers = layers
  
  def initialize(self, input_dim=None):
    dim = input_dim
    if dim is None and self.layers[0].Weight is not None:
      dim = self.layers[0].n
    assert dim is not None, 'Please provide input_dim'

    for layer in self.layers:
      if not layer.initialized:
        layer.initialize(dim)
      dim = layer.n

  def feedforward(self, X:nb.double[:, ::1]):
    A = X
    for layer in self.layers:
        A = layer.activate(A)
    return A

# nn = NyuNet2_dep(nb.typed.List([Dense_dep(8, input_dim=10), 
#                             Dense_dep(20),
#                             Dense_dep(1, activation='sigmoid')]))
# nn.initialize()
# X = np.random.rand(10, 5) * 2 - 1
# y_pred = nn.feedforward(X)
# y_pred

In [ ]:
#@title
spec = [
    ('x', nb.double[:]),
    ('v', nb.double[:]),
    ('v_max', nb.float32),
    ('phi', nb.typeof((.2, .2)))
]

@jitclass(spec)
class Particle:
  def __init__(self, dim, v_max:nb.float32=None, phi_1=.03, phi_2=.03, seed=None):
    if seed is not None:
      np.random.seed(seed)
    self.x = np.random.rand(dim)
    self.v = np.random.rand(dim)
    self.v_max = v_max
    self.phi = (phi_1, phi_2)  # learning rate
  
  def update(self):
    self.v = np.clip(self.v + self.phi[0] + np.random.sample(), 
                     a_min=-self.v_max, 
                     a_max=self.v_max)
    self.x = self.v + self.x
  
p1 = Particle(10, v_max=1.)
p1.update()
p1.x, p1.v;

# mungkin buat semua partikel jadi matriks saja.

In [ ]:
#@title
@jitclass([('X', nb.double[:, ::1]),
           ('V', nb.double[:, ::1]),
           ('P', nb.double[:, ::1]),
           ('v_max', nb.optional(nb.float32)),
           ('phi', nb.types.Tuple((nb.float32, nb.float32))),
           ('p_fit_', nb.double[::1]),
           ('best_fit', nb.float32)
          ])

class ParticleSwarm_dep:
  def __init__(self, dim, 
               n_particles=30,    # rekomendasi (Carlisle dan Dozier, 2001)
               v_max:nb.float32=None, 
               phi_1=2.8, phi_2=1.3,    # nilai phi atas rekomendasi (Carlisle dan Dozier, 2001)
               seed=None):
    if seed is not None:
      np.random.seed(seed)
    self.X = np.random.rand(n_particles, dim)   # matriks posisi
    self.V = np.random.rand(n_particles, dim)   # matriks kecepatan
    self.p_fit_ = np.zeros((n_particles,))  # vektor fitness terbaik tiap partikel 
    self.v_max = v_max    # skalar kecepatan maksimal
    self.P = self.X   # matriks posisi solusi terbaik tiap partikel
    self.best_fit = 0.    # skalar fitness global terbaik
    self.phi = (phi_1, phi_2)   # learning rate
  
  def step(self, fitness_func):
    # hitung fitness setiap partikel
    fitness = fitness_func(self.X)
    
    # update P partikel terbaik dan p_fitness terbaik
    tmp = np.stack((self.p_fit_, fitness), axis=1).T
    tmp = np_argmax_axis_0(tmp)

    for i, x in enumerate(tmp):
      if x == 1:
        self.P[i] = self.X[i]

    self.p_fit_ = np.maximum(fitness, self.p_fit_)
    
    # fitness terbaik secara global
    best_fitness_index = np.argmax(fitness)
    g_best_ = self.X[best_fitness_index]
    
    # update vektor kecepatan tiap partikel
    r = np.random.rand(self.X.shape[0], 1)
    self.V = self.V + \
          self.phi[0] * r * (self.P - self.X) + \
          self.phi[1] * r * (g_best_ - self.X)

    if self.v_max is not None:
      self.V = clip_np(self.V, a_min=-self.v_max, a_max=self.v_max)

    # update vektor posisi tiap partikel
    self.X = self.V + self.X